In [1]:
# Aplicação completa: Esta aplicação faz uso do tokenizador BPE (Byte Pair Encoding).

In [2]:
# Importa as bibliotecas necessárias para a aplicação.

In [3]:
import os  # Para manipulação de diretórios e arquivos.
import json  # Para carregar e manipular dados em formato JSON.
from tqdm import tqdm  # Exibe uma barra de progresso durante operações demoradas.
import sys  # Permite manipular parâmetros e caminhos do sistema.
from collections import defaultdict  # Cria dicionários com valor padrão automático.

sys.path.append(r"C:\Users\tagsa\Downloads")  # Adiciona ao sys.path o diretório onde o módulo BPE_Tokenizer pode estar localizado.
from bpe_tokenizer_otimizado_e_comentado import BPE_Tokenizer  # Importa o tokenizador BPE do módulo localizado no caminho adicionado.

In [7]:
# Inicializa o tokenizador BPE.

In [9]:
tokenizer = BPE_Tokenizer()

In [11]:
# Define o caminho para a pasta 'corpus', que deve conter os arquivos a serem processados (esta aplicaçao são arquivos JSON).

In [13]:
pasta_corpus = r'C:\Users\tagsa\Downloads\corpus'  # O 'r' antes da string indica uma raw string, preservando as barras invertidas.

In [15]:
def carregar_textos_json(pasta):
    """
    Carrega e combina todos os textos dos arquivos JSON em uma única string.

    Parâmetros:
    pasta : str
        Caminho para a pasta onde os arquivos JSON estão localizados.

    Retorna:
    tuple
        - String com o corpus combinado.
        - Número de arquivos JSON válidos lidos.
        - Contador de textos individuais encontrados.
    """
    corpus = []  # Lista para armazenar o conteúdo dos arquivos.
    arquivos_validos = 0  # Contador de arquivos JSON válidos.
    textos_encontrados = 0  # Contador de textos individuais.

    if not os.path.exists(pasta):
        print(f"Pasta não encontrada: {pasta}")
        return "", 0, 0  # Retorna valores padrão no caso de erro.

    # Lista os arquivos JSON encontrados na pasta.
    arquivos = [arq for arq in os.listdir(pasta) if arq.endswith(".json")]
    print(f"Número de arquivos JSON encontrados: {len(arquivos)}")

    # Itera sobre os arquivos e processa cada um.
    for arquivo in tqdm(arquivos, desc="Carregando arquivos JSON", unit="arquivo"):
        caminho = os.path.join(pasta, arquivo)
        try:
            with open(caminho, 'r', encoding='utf-8') as f:
                data = f.read()  # Lê o conteúdo do arquivo.
                corpus.append(data)  # Adiciona o conteúdo ao corpus.
                arquivos_validos += 1  # Incrementa o contador de arquivos válidos.
                textos_encontrados += len(data.split())  # Conta textos individuais.
        except (json.JSONDecodeError, OSError) as erro:
            print(f"Erro ao processar {arquivo}: {erro}")

    return " ".join(corpus), arquivos_validos, textos_encontrados  # Retorna a tupla.

In [17]:
# Contar a frequência de pares de tokens consecutivos em uma lista de IDs.

In [19]:
def get_most_frequent_pairs(ids):
    """
    Calcula a frequência de pares consecutivos de tokens na lista de IDs.

    Parâmetros:
    ids : list of list of int
        Lista contendo sequências de IDs de tokens.

    Retorna:
    dict
        Dicionário com pares de tokens como chave e suas frequências como valor.
    """
    pairs = defaultdict(int)  # Inicializa um dicionário com valor padrão 0 para cada par.

    # Itera sobre cada sequência de IDs na lista.
    for id_seq in ids:
        # Itera sobre os índices da sequência para formar pares consecutivos.
        for i in range(len(id_seq) - 1):
            pair = (id_seq[i], id_seq[i + 1])  # Cria um par com dois tokens consecutivos.
            pairs[pair] += 1  # Incrementa a contagem do par no dicionário.

    return pairs  # Retorna o dicionário de pares e suas frequências.

In [21]:
# Função para realizar a mesclagem dos pares mais frequentes.

In [23]:
# Realizar a mesclagem dos pares de tokens mais frequentes até um máximo de x.
def merge_pairs(ids, tokenizar, limite=256):
    """
    Realiza a mesclagem dos pares mais frequentes com base no limite fornecido.

    Parâmetros:
    ids : list of list of int
        Lista de IDs a serem processados.
    tokenizar : objeto
        Objeto que contém métodos de tokenização.
    limite : int, opcional
        Limite de pares a serem mesclados (padrão: 20).

    Retorna:
    list of list of int
        Lista de IDs atualizada após as mesclagens.
    """
    # Obtém as estatísticas dos pares de tokens.
    status = tokenizar.get_stats(ids)
    pares_ordenados = sorted(status.items(), key=lambda x: x[1], reverse=True)[:limite]

    global merge_count  # Garante que estamos usando o contador global.

    for (a, b), freq in pares_ordenados:
        # Obtém os caracteres correspondentes aos IDs.
        token_a = tokenizar.vocab.get(a, b'\x7f').decode('utf-8', errors='replace')
        token_b = tokenizar.vocab.get(b, b'\x7f').decode('utf-8', errors='replace')

        # Concatena os tokens para exibir o novo token.
        novo_token = f"{token_a}{token_b}"

        # Exibe o output no formato desejado.
        print(f"Mesclando ({a}, {b}) ({token_a}, {token_b}) em um novo token {merge_count} ({novo_token}) com frequência {freq}")

        # Realiza a mesclagem e incrementa o contador de tokens.
        ids = tokenizar.merge(ids, (a, b), merge_count)
        merge_count += 1  # Incrementa o contador após a mesclagem.

    return ids  # Retorna a lista de IDs atualizada.

In [25]:
# Carrega e combina todos os textos dos arquivos JSON em uma única string.

In [27]:
corpus_texto, arquivos_validos, textos_encontrados = carregar_textos_json(pasta_corpus)

Número de arquivos JSON encontrados: 10000


Carregando arquivos JSON: 100%|████████████████████████████████████████████| 10000/10000 [01:02<00:00, 159.21arquivo/s]


In [28]:
# Exibe no console os resultados do processamento, fornecendo informações sobre o corpus carregado.

In [29]:
print(f"Número de arquivos JSON válidos lidos: {arquivos_validos}")  # Exibe as informações sobre o corpus carregado.
print(f"Corpus carregado com {len(corpus_texto)} caracteres.")  # Exibe o total de caracteres no corpus carregado.
print(f"Número de textos individuais: {textos_encontrados}")  # Exibe o número total de textos individuais encontrados.

Número de arquivos JSON válidos lidos: 10000
Corpus carregado com 71738184 caracteres.
Número de textos individuais: 11555410


In [30]:
# Treinamento do tokenizador BPE.

In [31]:
tokenizer.train(corpus_texto, vocab_size=512)

Treinamento do tokenizador: 100%|████████████████████████████████████████████| 256/256 [1:17:48<00:00, 18.24s/iteração]


Treinamento do tokenizador BPE concluído!


In [32]:
# Lista de mesclagem.

In [33]:
merge_count = 256  # Inicializa um contador para os novos tokens.
ids = list(corpus_texto.encode("utf-8"))  # Converte o corpus para uma lista de IDs.
ids = merge_pairs(ids, tokenizer)  # Realiza a mesclagem dos 20 pares mais frequentes (conforme definido).

Mesclando (111, 32) (o,  ) em um novo token 256 (o ) com frequência 1936188
Mesclando (97, 32) (a,  ) em um novo token 257 (a ) com frequência 1768162
Mesclando (101, 32) (e,  ) em um novo token 258 (e ) com frequência 1692138
Mesclando (115, 32) (s,  ) em um novo token 259 (s ) com frequência 1397165
Mesclando (32, 100) ( , d) em um novo token 260 ( d) com frequência 1374945
Mesclando (100, 101) (d, e) em um novo token 261 (de) com frequência 1071607
Mesclando (32, 101) ( , e) em um novo token 262 ( e) com frequência 831685
Mesclando (44, 32) (,,  ) em um novo token 263 (, ) com frequência 799147
Mesclando (114, 97) (r, a) em um novo token 264 (ra) com frequência 770498
Mesclando (101, 115) (e, s) em um novo token 265 (es) com frequência 769312
Mesclando (32, 97) ( , a) em um novo token 266 ( a) com frequência 765729
Mesclando (100, 111) (d, o) em um novo token 267 (do) com frequência 737449
Mesclando (111, 115) (o, s) em um novo token 268 (os) com frequência 683818
Mesclando (32, 112

In [34]:
# Exibe estatísticas finais.

In [35]:
print(f"\nComprimento final dos tokens: {len(corpus_texto)}")  # Calcula o número total de caracteres presentes no corpus original após o processo de tokenização.
print(f"Comprimento final dos IDs: {len(ids)}")  # Exibe a quantidade de IDs (tokens únicos) gerados após a aplicação do tokenizador.


Comprimento final dos tokens: 71738184
Comprimento final dos IDs: 45218409


In [36]:
# Calcula e exibe a taxa de compressão.

In [47]:
taxa_compressao = len(corpus_texto) / len(ids)  # Divide o comprimento do texto original pelo número de IDs para indicar a eficiência da tokenização.
print(f"Taxa de compressão: {taxa_compressao:.2f}X")  # Formata a saída para exibir a taxa com duas casas decimais

Taxa de compressão: 1.59X
